## 폐암 수술 환자의 생존율 데이터 분석

In [1]:
# 필요한 라이브러리
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# 실행할 때마다 같은 결과를 출력하기 위한 seed 값 설정
seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

In [3]:
# 준비된 수술 환자 데이터를 불러들임
Data_set = np.loadtxt("dataset/ThoraricSurgery.csv", delimiter=",")
Data_set

array([[293.  ,   1.  ,   3.8 , ...,   0.  ,  62.  ,   0.  ],
       [  1.  ,   2.  ,   2.88, ...,   0.  ,  60.  ,   0.  ],
       [  8.  ,   2.  ,   3.19, ...,   0.  ,  66.  ,   1.  ],
       ...,
       [406.  ,   6.  ,   5.36, ...,   0.  ,  62.  ,   0.  ],
       [ 25.  ,   8.  ,   4.32, ...,   0.  ,  58.  ,   1.  ],
       [447.  ,   8.  ,   5.2 , ...,   0.  ,  49.  ,   0.  ]])

In [4]:
# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장
x_data = np.array(Data_set[:, 0:17], dtype=np.float64)
y_data = np.array(Data_set[:, 17], dtype=np.float64).reshape(470, 1)

In [5]:
print(x_data.shape, y_data.shape)

(470, 17) (470, 1)


In [66]:
# 입력 값을 플레이스 홀더에 저장
X = tf.placeholder(tf.float64, shape=[None, 17])
Y = tf.placeholder(tf.float64, shape=[None, 1])

In [67]:
# Layer 1의 웨이트 W와 바이어스 b의 값을 임의로 정함 
W1 = tf.Variable(tf.random_normal([17,30], dtype=tf.float64)) 
b1 = tf.Variable(tf.random_normal([30], dtype=tf.float64))
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

### 아래와 같이 하면 cost 값이 nan 이 됨

In [68]:
# Layer 2
W2 = tf.Variable(tf.random_normal([30,1], dtype=tf.float64)) 
b2 = tf.Variable(tf.random_normal([1], dtype=tf.float64))
hypo = tf.sigmoid(tf.add(tf.matmul(L1, W2), b2))

In [69]:
# Binary Cross Entropy
learning_rate = 0.1
cost = tf.reduce_mean(-Y * tf.log(hypo) - (1.-Y) * tf.log(1.-hypo))

In [71]:
predict = tf.cast(hypo > 0.5, dtype=tf.float64)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.float64))

In [73]:
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [74]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(10):
    _, cost_val, acc_val = sess.run([train, cost, accuracy], feed_dict = {X: x_data, Y: y_data})
    print("step=%d, cost_val=%.5f, accuracy: %.4f" % (epoch + 1, cost_val, acc_val))

step=1, cost_val=nan, accuracy: 0.8106
step=2, cost_val=nan, accuracy: 0.8511
step=3, cost_val=nan, accuracy: 0.8511
step=4, cost_val=nan, accuracy: 0.8511
step=5, cost_val=nan, accuracy: 0.8511
step=6, cost_val=nan, accuracy: 0.8511
step=7, cost_val=nan, accuracy: 0.8511
step=8, cost_val=nan, accuracy: 0.8511
step=9, cost_val=nan, accuracy: 0.8511
step=10, cost_val=nan, accuracy: 0.8511


### nan을 피하려면 다음과 같이 하여야 함

In [76]:
# Layer 2
W2 = tf.Variable(tf.random_normal([30,1], dtype=tf.float64)) 
b2 = tf.Variable(tf.random_normal([1], dtype=tf.float64))
hypo = tf.add(tf.matmul(L1, W2), b2)

In [77]:
# Binary Cross Entropy
learning_rate = 0.1
cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=hypo, labels=Y)

In [78]:
predict = tf.cast(hypo > 0, dtype=tf.float64)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.float64))

In [79]:
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [80]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(10):
    _, cost_val, acc_val = sess.run([train, cost, accuracy], feed_dict = {X: x_data, Y: y_data})
    print("step=%d, cost_val=%.5f, accuracy: %.4f" % (epoch + 1, cost_val[0], acc_val))

step=1, cost_val=2042.35160, accuracy: 0.1489
step=2, cost_val=1398.13367, accuracy: 0.1489
step=3, cost_val=799.63262, accuracy: 0.2277
step=4, cost_val=250.05682, accuracy: 0.3936
step=5, cost_val=0.00000, accuracy: 0.8511
step=6, cost_val=0.00000, accuracy: 0.8511
step=7, cost_val=0.00000, accuracy: 0.8511
step=8, cost_val=0.00000, accuracy: 0.8511
step=9, cost_val=0.00000, accuracy: 0.8511
step=10, cost_val=0.00000, accuracy: 0.8511


### 평가

In [81]:
acc = sess.run(accuracy, feed_dict = {X: x_data, Y: y_data})
print("Accuracy: %.4f" % acc)

Accuracy: 0.8511


### 분석

In [82]:
W1_, W2_, b1_, b2_ = sess.run([W1, W2, b1, b2])

In [89]:
def ReLU(x):
    return x * (x > 0)
def sigmoid(x):
    return 1./(1. + np.e**(-x))

In [84]:
b1_

array([-0.60944192,  0.30364601,  0.04540281, -0.54825976, -0.77957685,
       -0.565919  ,  1.54174703, -0.5864513 , -1.12972836,  0.83256405,
       -1.10162785, -2.24775087, -1.0463446 , -0.99829141, -0.77690056,
       -1.62534383, -1.66153152,  0.07146427, -1.25389784, -0.31012793,
        0.55201878, -1.06460855,  0.0973948 , -4.26803179, -0.30713957,
       -1.91532796,  0.53727522, -1.01371593,  1.60610197,  2.2275901 ])

In [87]:
L1_ = ReLU(np.matmul(x_data[0,:].reshape(1,17), W1_) + b1_)
L1_

array([[ -0.        ,  -0.        ,  -0.        ,  -0.        ,
         15.3905051 , 167.81089366,  -0.        , 283.60536853,
        191.58026911,  -0.        ,  11.18896191,  -0.        ,
         -0.        ,  -0.        ,  -0.        ,  -0.        ,
         82.56645688, 762.93011118,  -0.        , 380.1102242 ,
         -0.        ,  -0.        ,  -0.        ,  -0.        ,
         -0.        ,  -0.        ,  98.34325327,  -0.        ,
         -0.        ,  26.31662324]])

In [95]:
L2_ = np.matmul(L1_, W2_) + b2_
H = sigmoid(L2_)
print(L2_[0].item(), H[0].item())

-1514.5824686985181 0.0


C:\Users\ckkim\.conda\envs\tf2\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in power
  after removing the cwd from sys.path.


In [96]:
y_data[0].item()

0.0